<a href="https://colab.research.google.com/github/KczBen/tol403-lokaverkefni/blob/main/Lokaverkefni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/KczBen/tol403-lokaverkefni.git

Cloning into 'tol403-lokaverkefni'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 489.62 KiB | 6.80 MiB/s, done.


In [3]:
import pandas as pd

nodes = pd.read_csv('tol403-lokaverkefni/data/nodes.tsv', sep='\t')
edges = pd.read_csv('tol403-lokaverkefni/data/edges.tsv', sep='\t')